In [28]:
from git import Repo

In [29]:
repo_path = "/Users/ashegde/Documents/projs/genai/experiments-genai/langchain/data/code"
# repo = Repo.clone_from("https://github.com/beaver-infra/template-py-rest-microservice", to_path=repo_path)

In [30]:
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language

print(repo_path)
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500),
)
documents = loader.load()
len(documents)

/Users/ashegde/Documents/projs/genai/experiments-genai/langchain/data/code


18

In [32]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap=200
)
texts = python_splitter.split_documents(documents)
print(len(texts))
print(texts)

16
[Document(page_content='"""\nContains service level configurations and metadata retrieval methods.\n"""\n\nfrom typing import ClassVar\n\nfrom pydantic_settings import BaseSettings', metadata={'source': '/Users/ashegde/Documents/projs/genai/experiments-genai/langchain/data/code/app/metadata.py', 'language': <Language.PYTHON: 'python'>}), Document(page_content='class Metadata(BaseSettings):\n    """\n    Represents metadata for the application.\n    """\n\n    # Metadata properties\n    title: ClassVar[str] = "dummy_users"\n    description: ClassVar[str] = (\n        "Service to fetch the list of dummy users from 3rd party downstream services"\n    )\n    version: ClassVar[str] = "1.1.6"\n    contact: ClassVar[dict] = {\n        "name": "Ashwin Hegde",\n        "url": "https://github.com/hegdeashwin",\n        "email": "ashwin.hegde3@gmail.com",\n    }\n\n    def get_service_title(self):\n        """\n        Retrieves the title of the service.\n        """\n        return self.title

In [34]:
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5",     # Provide the pre-trained model's path
    model_kwargs={'device':'cpu'}, # Pass the model configuration options
    encode_kwargs={'normalize_embeddings': True} # Pass the encoding options
)

print(embeddings)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='BAAI/bge-large-en-v1.5' cache_folder=None model_kwargs={'device': 'cpu'} encode_kwargs={'normalize_embeddings': True} multi_process=False show_progress=True


In [35]:
db = Chroma.from_documents(texts, embedding=embeddings)
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 8},
)

Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:21<00:00, 21.41s/it]


In [36]:
from langchain_community.chat_models.ollama import ChatOllama
llm = ChatOllama(model="codellama")

In [37]:
response_message = llm.invoke(
    "what the healthcheck REST API endpoint"
)

print(response_message.content)
print(response_message.response_metadata)


The healthcheck REST API endpoint is typically used to check the status of a service or application. When making a request to this endpoint, the service or application will return some form of response that indicates its current state and readiness to handle requests. This can be useful for monitoring and debugging purposes, as well as for ensuring that the service or application is available and functioning correctly.

Some common examples of responses from a healthcheck REST API endpoint include:

* "OK" or "200": The service or application is up and running, and ready to handle requests.
* "Error" or "500": There is an error with the service or application that prevented it from responding to the request.
* "Service Unavailable" or "503": The service or application is currently unavailable due to maintenance, high traffic, or other reasons.
* "Not Implemented" or "501": The requested method or endpoint is not implemented by the service or application.
* "Bad Request" or "400": The 

In [38]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_template = """
Answer the user's questions based on the below context.
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible:

{context}
"""

# First we need a prompt that we can pass into an LLM to generate this search query
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("user", "{input}"),
    ]
)
document_chain = create_stuff_documents_chain(llm, prompt)

qa_chain = create_retrieval_chain(retriever, document_chain)

In [39]:
# Run, only returning the value under the answer key for readability
qa_chain.pick("answer").invoke({"input": "How to access health check API?"})

Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 11.45it/s]


'You can access the health check API by making a GET request to <http://localhost:8000/api/v1/health>. The response should be JSON containing the status of your API, which in this case is `true`.'